In [54]:
import pandas as pd 
pd.options.mode.chained_assignment = None  # default='warn'

In [55]:
data = pd.read_csv('data\prix-carburants-fichier-quotidien-test-ods.csv', sep= ';', on_bad_lines= 'skip')

In [56]:
data.columns

Index(['id', 'cp', 'pop', 'adresse', 'ville', 'horaires', 'rupture',
       'fermeture', 'geom', 'prix_maj', 'prix_id', 'prix_valeur', 'prix_nom',
       'com_arm_code', 'Nom Officiel Commune / Arrondissement Municipal',
       'Code Officiel EPCI', 'Nom Officiel EPCI', 'Code Officiel Département',
       'Nom Officiel Département', 'Code Officiel Région',
       'Nom Officiel Région', 'services_service', 'Nom rupture',
       'Début rupture', 'Fin rupture', 'Automate 24-24 (oui/non)'],
      dtype='object')

Fonction pour définir la "bonne" base de données avec les données qui nous intéressent  
Il faut l'améliorer pour qu'elle élimine les doublons

In [57]:
def data_carb(df, carb):
    data_1 = df[['adresse', 'ville', 'geom', 'prix_valeur', 'prix_nom']]
    d = data_1[data_1.prix_nom == carb]
    d.drop_duplicates(keep = 'first', inplace=True) 
    #df.drop_duplicates(subset ="'column'", keep = 'first', inplace=True) si on veut supprimer selon une colonne
    return d




In [58]:
data_2 = data_carb(data, 'SP95').sort_values('prix_valeur')

In [59]:
data_2

,adresse,ville,geom,prix_valeur,prix_nom
2601,Avenue de l'Orme à Martin,Courcouronnes,"48.631,2.417",1.169,SP95
2737,18 Grande Rue,DOZULé,"49.231,-0.044",1.290,SP95
65471,13 Avenue Ampère,MONTIGNY-LE-BRETONNEUX,"48.798,2.048",1.423,SP95
10380,le bourg,Groléjac,"44.820868467717006,1.2905781261784",1.473,SP95
14519,531 Route de Strasbourg,VIRIAT,"46.236,5.263",1.481,SP95
...,...,...,...,...,...
8063,23 Boulevard Victor Hugo,Saint-Ouen,"48.911,2.332",2.290,SP95
24100,"8,10,10bis Rue Bailleul",PARIS,"48.861,2.341",2.370,SP95
11249,15 Avenue Duquesne,PARIS,"48.853,2.308",2.600,SP95
43085,69 Avenue Kléber,Paris,"48.865,2.289",2.690,SP95


Fonction pour calculer la distance à partir de données de longitude et de latitude

In [60]:
from math import cos, sin, acos 

def distance1(p1,p2):
    lat1,long1 = p1
    lat2,long2 = p2
    dlong = long2 - long1
    dist_ang = acos(sin(lat1)*sin(lat2) + cos(lat1)*cos(lat2)*cos(dlong))
    return(6378137*dist_ang)

In [61]:
from geopy.geocoders import Nominatim


def Adresse_to_gps(Adresse) : 

    geolocator = Nominatim(user_agent="Projet Python")
    location = geolocator.geocode(Adresse)
    return str(str(location.latitude) + ','+ str(location.longitude))


Adresse_to_gps("1 rue d'Arsonval, Orsay, France")

'48.7087952,2.1742033'

Fonction pour renvoyer un data frame avec les 10 plus proches stations essences qui propose le type de carburant recherché   
Majeur problème : les doublons 

In [62]:
def dix_plus_proche_card(df, posi, carb):
    data_1 = data_carb(df, carb).sort_values('prix_valeur')
    pos = list(map(float,Adresse_to_gps(posi).split(',')))
    def distancetoposi(p2):
        pos_carb = list(map(float, p2.split(',')))
        return distance1(pos, pos_carb)
    new_df = data_1['geom'].apply(distancetoposi).sort_values().head(10)
    return data_1.loc[new_df.index.to_list()]



    


In [65]:
dix_plus_proche_card(data, "1 rue d'Arsonval, Orsay, France" , 'SP95')

,adresse,ville,geom,prix_valeur,prix_nom
18723,69 Avenue Charles de Gaulle,MORANGIS,"48.706,2.343",1.759,SP95
43435,Rue de la Ballastière,Ligny-en-Barrois,"48.677,5.326",1.700,SP95
38126,180 ROUTE NATIONALE 7,ATHIS-MONS,"48.712,2.371",1.709,SP95
50277,KERGUENNEC,ROSCOFF,"48.71344,-3.98569",1.750,SP95
5332,lieudit le village,Les Essarts-le-Roi,"48.71419,1.92645",1.745,SP95
10514,RUE DU GENERAL DE GAULLE,AVRANCHES,"48.68647,-1.36356",1.819,SP95
46246,PLACE DE BEAUPLAN,SAINT-RéMY-LèS-CHEVREUSE,"48.716,2.085",1.737,SP95
18710,72 Avenue Jean Jaurès,Montgeron,"48.716,2.446",1.770,SP95
13697,118 Rue de la Passerelle,CHILLY-MAZARIN,"48.69943,2.32057",1.799,SP95
18347,120A ROUTE NATIONALE,GAMBSHEIM,"48.699248714218,7.889266294415299",1.589,SP95
